In [1]:
import os
import sys
import joblib
import numpy as np 
# from tqdm import tqdm_notebook as tqdm
sys.path.append('../')
import json
from zipfile import ZipFile, ZIP_DEFLATED
# from util import test_spearmanr

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from torch.utils.data import DataLoader

%load_ext autoreload
%autoreload 2
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import scipy 
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

def test_spearmanr(vid_embedding, annotation_file):
    relevances, similarities = [], []
    with open(annotation_file, 'r') as f:
        for line in tqdm(f):
            query, candidate, relevance = line.split()
            if query not in vid_embedding:
                print(f'ERROR: {query} NOT found')
                continue
                # raise Exception(f'ERROR: {query} NOT found')
            if candidate not in vid_embedding:
                print(f'ERROR: {candidate} NOT found')
                continue
                # raise Exception(f'ERROR: {candidate} NOT found')
            # print('pass')
            query_embedding = vid_embedding.get(query)
            candidate_embedding = vid_embedding.get(candidate)
            similarity = cosine_similarity([query_embedding], [candidate_embedding])[0][0]
            similarities.append(similarity)
            relevances.append(float(relevance))

    spearmanr = scipy.stats.spearmanr(similarities, relevances).correlation
    print('spearmanr:', spearmanr)
    return spearmanr
annotation_file = '/data03/yrqUni/Workspace/QQB/Data/pairwise/label.tsv'


In [2]:
# yrq_valid
import pickle
with open('/data03/yrqUni/Workspace/QQB/yrq/VAl.pickle', 'rb') as file:
    yrq_bert_valid = pickle.load(file)
    
with open('/data03/yrqUni/Workspace/QQB/yrq/TEST.pickle', 'rb') as file:
    yrq_bert_test = pickle.load(file)
    
with open('/data03/yrqUni/Workspace/QQB/yrq/VAl_2.pickle', 'rb') as file:
    yrq_roberta_valid = pickle.load(file)
    
with open('/data03/yrqUni/Workspace/QQB/yrq/TEST_2.pickle', 'rb') as file:
    yrq_roberta_test = pickle.load(file)
    
yrq_bert_valid = [yrq_bert_valid]
yrq_bert_test = [yrq_bert_test]
yrq_roberta_valid = [yrq_roberta_valid]
yrq_roberta_test =[yrq_roberta_test]

In [3]:
!du -sh /data03/yrqUni/Workspace/QQB/yrq/*

93M	/data03/yrqUni/Workspace/QQB/yrq/TEST_2.pickle
93M	/data03/yrqUni/Workspace/QQB/yrq/TEST.pickle
130M	/data03/yrqUni/Workspace/QQB/yrq/VAl_2.pickle
130M	/data03/yrqUni/Workspace/QQB/yrq/VAl.pickle


In [4]:
import json

def json_loads(path):
    with open(path, 'r') as load_f:
        load_dict = json.load(load_f)
    for k in load_dict:
        load_dict[k] = np.array(load_dict[k])
    return load_dict

yrq_valid = [json_loads('/data03/yrqUni/Workspace/QQB/zp/val_emd.json')]
yrq_test = [json_loads('/data03/yrqUni/Workspace/QQB/zp/result.json')]



In [5]:
# def trans2array(inp):
#     ans = []
#     for emb in inp:
#         for k in emb:
#             emb[k] = np.array(emb[k]).astype(np.float16)
#         ans.append(emb)
#     return ans

# yrq_bert_valid = trans2array(yrq_bert_valid)
# yrq_bert_test = trans2array(yrq_bert_test)
# yrq_roberta_valid = trans2array(yrq_roberta_valid)
# yrq_roberta_test = trans2array(yrq_roberta_test)


In [6]:
ich_valid = joblib.load('/data03/yrqUni/Workspace/QQB/zp/ICH_valid.pkl')
ich_test = joblib.load('/data03/yrqUni/Workspace/QQB/zp/ICH_test.pkl')



In [7]:
valid_two_stage_file = '/data03/yrqUni/Workspace/QQB/zp/code/model2/pkl/test_bitm_vision_transdp01_bertlast_valid.pkl'
test_two_stage_file = '/data03/yrqUni/Workspace/QQB/zp/code/model2/pkl/test_bitm_vision_transdp01_bertlast_test.pkl'

baseline_valid = joblib.load(valid_two_stage_file)
baseline_test = joblib.load(test_two_stage_file)

valid_two_stage_file2 = '/data03/yrqUni/Workspace/QQB/zp/code/model2/pkl/test_robitm_vision_transdp01_bertlast_valid.pkl'
test_two_stage_file2 = '/data03/yrqUni/Workspace/QQB/zp/code/model2/pkl/test_robitm_vision_transdp01_bertlast_test.pkl'

baseline_valid2 = joblib.load(valid_two_stage_file2)
baseline_test2 = joblib.load(test_two_stage_file2)

In [8]:
zlh_trans_valid = joblib.load('/data03/yrqUni/Workspace/QQB/ZLH/pretrain_baseline/pkl/valid_two_stagedict_input_zlh1002.pkl')
zlh_trans_test = joblib.load('/data03/yrqUni/Workspace/QQB/ZLH/pretrain_baseline/pkl/test_two_stagedict_input_zlh1002.pkl')


In [9]:
roberta_valid = joblib.load('/data03/yrqUni/Workspace/QQB/zp/code/baseline/pkl/roberta_valid_seq.pkl')
roberta_test = joblib.load('/data03/yrqUni/Workspace/QQB/zp/code/baseline/pkl/roberta_test_seq.pkl')

In [10]:
valid_knowledgePool_t = baseline_valid + roberta_valid + zlh_trans_valid \
    + baseline_valid2 + ich_valid + yrq_valid + yrq_bert_valid + yrq_roberta_valid
test_knowledgePool_t = baseline_test + roberta_test + zlh_trans_test \
    + baseline_test2 + ich_test + yrq_test + yrq_bert_test + yrq_roberta_test
 
# knowledgePool is list of dict 
'''
every dict like:
{
    "vid0":np.array([1,2,3])...
}
'''


'\nevery dict like:\n{\n    "vid0":np.array([1,2,3])...\n}\n'

In [11]:

valid_knowledgePool = []
test_knowledgePool = []
print(len(valid_knowledgePool_t))
for k1,k2 in zip(valid_knowledgePool_t, test_knowledgePool_t):
    if len(k1['2345203561710400875'].shape) == 1:
#         score = test_spearmanr(k1, annotation_file)
#         print(k1['2345203561710400875'].shape, score)
#         if score < 0.6:
#             print("continue")
#             continue
        valid_knowledgePool.append(k1)
        test_knowledgePool.append(k2)

for k1, k2 in zip(valid_knowledgePool_t, test_knowledgePool_t):
    if len(k1['2345203561710400875'].shape) == 2:
        print(k1['2345203561710400875'].shape)
        valid_knowledgePool.append(k1)
        test_knowledgePool.append(k2)
del valid_knowledgePool_t, test_knowledgePool_t
import gc
gc.collect()

28
(32, 768)
(32, 1024)
(32, 1024)


46

In [12]:
for i in valid_knowledgePool[0]:
    print(i)
    break

2345203561710400875


In [13]:
inputszlist = [know['2345203561710400875'].shape for know in valid_knowledgePool]
print(inputszlist)


[(256,), (256,), (256,), (10000,), (512,), (1024,), (1024,), (10000,), (10000,), (512,), (768,), (768,), (768,), (512,), (512,), (512,), (10000,), (256,), (256,), (256,), (10000,), (256,), (256,), (512,), (512,), (32, 768), (32, 1024), (32, 1024)]


In [14]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
# from torchvision import datasets
from torchvision.transforms import ToTensor

class ValidDataSet(Dataset):
    def __init__(self, knowledgePool, df, valid=True):
        self.df = df
        self.knowledgePool = knowledgePool

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        line = self.df.iloc[idx]
        query, candidate, relevance = line['query'], line["candidate"], line["relevance"]

        queryemblist = []
        for knowledge in self.knowledgePool:
            queryemblist.append(torch.from_numpy(knowledge[query].astype(np.float32)))

        candidateemblist = []
        for knowledge in self.knowledgePool:
            candidateemblist.append(torch.from_numpy(knowledge[candidate].astype(np.float32)))

        relevance = float(relevance)
        return  tuple(queryemblist), tuple(candidateemblist), relevance
            

class testDataSet(Dataset):
    def __init__(self, knowledgePool, valid=True):
        self.knowledgePool = knowledgePool
        self.id = [id for id in self.knowledgePool[0]]

    def __len__(self):
        return len(self.id)

    def __getitem__(self, idx):
        id = self.id[idx]
        emblist = []
        for knowledge in self.knowledgePool:
            emblist.append(torch.from_numpy(knowledge[id].astype(np.float32)))
            
        return emblist, id
            
        

In [15]:
query_list = []
candidate_list = []
relevance_list = []
data_path = '/data03/yrqUni/Workspace/QQB/Data'
annotation_file = os.path.join(data_path, 'pairwise/label.tsv')

with open(annotation_file, 'r') as f:
    for line in f:
        query, candidate, relevance = line.split()
        query_list.append(query)
        candidate_list.append(candidate)
        relevance_list.append(relevance)


valid_df = pd.DataFrame({
    "query":query_list,
    "candidate":candidate_list,
    "relevance":relevance_list
})

valid_df['relevance'] =  valid_df['relevance'].apply(float)
print(valid_df['relevance'].value_counts(normalize=True))
valid_df.head()

0.00    0.363658
1.00    0.139192
0.50    0.069486
0.05    0.055126
0.10    0.038160
0.60    0.030648
0.15    0.030428
0.55    0.028866
0.65    0.028410
0.35    0.023240
0.40    0.023152
0.45    0.020148
0.85    0.019367
0.20    0.019249
0.90    0.019161
0.95    0.017703
0.70    0.017555
0.30    0.016583
0.25    0.015582
0.80    0.012592
0.75    0.011694
Name: relevance, dtype: float64


,query,candidate,relevance
0,2345203561710400875,5344606281729891758,1.00
1,759945328762912122,5560784129870034298,0.85
2,2336209090533711226,8010747189096226170,1.00
3,2588403135132530094,5380633252649788846,0.25
4,3245934094259209594,120437073895839098,1.00


In [16]:
BATCHSIZETEST = 3072
testdataset = testDataSet(test_knowledgePool)
test_dataloader = DataLoader(testdataset, batch_size=BATCHSIZETEST, shuffle=False, drop_last=False)

In [17]:
from sklearn.model_selection import train_test_split
# ?train_test_split

from KN_TEXTCNN import Network

# inputszlist = [k[0] for k in inputszlist]

import scipy
from torch.nn.functional import normalize, dropout

from customloss import *
from termcolor import colored
from sklearn.model_selection import train_test_split
TEMP = 0.2
ALLIN = False
OUTSZ = 256
LOSS = SpearmanCorrelationLoss(TEMP) 
# LOSS = WhiteMSE()
# LOSS = wiseMSE()
# LOSS = SentenceBertLoss(OUTSZ).cuda()
print(LOSS)
DEVICE = 'cuda:0'
BATCHSIZE = int(2048*0.7)
print(len(inputszlist), inputszlist)

from sklearn.model_selection import GroupKFold
np.random.seed(42)
group_kfold = GroupKFold(n_splits=5)
# train_part, valid_part = train_test_split(valid_df, test_size=0.01)

y = valid_df['relevance']
groups = valid_df['candidate']
train_part, valid_part = 0, 0 


# test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

SpearmanCorrelationLoss()
28 [(256,), (256,), (256,), (10000,), (512,), (1024,), (1024,), (10000,), (10000,), (512,), (768,), (768,), (768,), (512,), (512,), (512,), (10000,), (256,), (256,), (256,), (10000,), (256,), (256,), (512,), (512,), (32, 768), (32, 1024), (32, 1024)]


In [18]:
from tqdm.notebook import tqdm
def query_expansion(df):
    '''
    1. (A, B) = 1 (B, C) = 1 ==> (A, C) = 1
    2. (A, B) = 1 (B, C) = 0 ==> (A, C) = 0
    '''
    
    relevance_df = df[df['relevance'] == 1.0]
    print('relevance_df:', relevance_df.shape)
    notrelevance_df = df[df['relevance'] == 0.0]
    print('notrelevance_df:', notrelevance_df.shape)
    # (A, B) = 1 (B, C) = 1 ==> (A, C) = 1
    query_list = []
    candidate_list = []
    relevance_list = []
    
    rule1_count = 0
    for A in tqdm(relevance_df.candidate.values):
        # find very B which (A, B) = 1
        B_set = set(relevance_df[relevance_df['candidate'] == A]["query"]) | \
            set(relevance_df[relevance_df['query'] == A]["candidate"])
        
        for B in B_set:
            # find very C which (B, C) = 1
            C_set = set(relevance_df[relevance_df['candidate'] == B]['query']) | \
                set(relevance_df[relevance_df['query'] == B]['candidate'])
            for C in C_set:
                if C != A:
                    rule1_count += 1
                    query_list.append(C)
                    candidate_list.append(A)
                    relevance_list.append(1.0)
    print('rule1_count:', rule1_count)
    rule1_df = pd.DataFrame({
        "query":query_list,
        "candidate":candidate_list,
        "relevance":relevance_list
    })
    query_list = []
    candidate_list = []
    relevance_list = []
    
    rule2_count = 0
    for A in tqdm(relevance_df.candidate.values):
        # find very B which (A, B) = 1
        B_set = set(relevance_df[relevance_df['candidate'] == A]["query"]) | \
            set(relevance_df[relevance_df['query'] == A]["candidate"])
        for B in B_set:
            # find very C which (B, C) = 0
            C_set = set(notrelevance_df[notrelevance_df['candidate'] == B]['query']) | \
                set(notrelevance_df[notrelevance_df['query'] == B]['candidate'])
            for C in C_set:
                if C != A:
                    rule2_count += 1
                    query_list.append(C)
                    candidate_list.append(A)
                    relevance_list.append(0.0)
    print('rule2_count:', rule2_count)
    rule2_df = pd.DataFrame({
        "query":query_list,
        "candidate":candidate_list,
        "relevance":relevance_list
    })
    valid_df = pd.concat([rule1_df.sample(30000), rule2_df, df], axis=0)#.sample(fraq=1)
    return valid_df
            
                


In [19]:
# testing




def inference_one_epoch(test_dataloader, NET):
    vid_embedding = {}
    NET.eval()
    for data1, ids in test_dataloader:
        data1 = tuple([data.to(DEVICE) for data in data1])
        with torch.no_grad():
            out1 = NET(data1).detach().cpu().numpy().astype(np.float16)
        for vid, embedding in zip(ids, out1):
            vid_embedding[vid] = embedding.tolist()
    return vid_embedding


def merge_embedding(vid_embedding_list):
    info = f'total {len(vid_embedding_list)} embeddings'
    print(colored(info, 'blue'))
    embedding_number = len(vid_embedding_list)
    final_embedding = {}
    all_vids = [key for key in vid_embedding_list[0]] 
    from tqdm import tqdm
    for vid in tqdm(all_vids):
        ans = 0
        for embedding in vid_embedding_list:
            ans += np.array(embedding[vid]) #/ embedding_number
        final_embedding[vid] = ans.astype(np.float16).tolist()
    return final_embedding



In [20]:
# NET = Network(len(inputszlist), inputszlist, hiddensz=1536, outsz=OUTSZ).to(DEVICE)
# print(NET)
vid_embedding_list = []
EPOCH = 8
START_INFERTEN_EPOCH = 5

for train_index, test_index in group_kfold.split(valid_df, y, groups):
    # print("TRAIN:", train_index, "TEST:", test_index)
    train_part, valid_part = valid_df.iloc[train_index], valid_df.iloc[test_index]
#     train_part = query_expansion(train_part)
    print(train_part.shape)
    print(valid_part.shape)

    training_data = ValidDataSet(valid_knowledgePool, train_part)
    validing_data = ValidDataSet(valid_knowledgePool, valid_part)


    train_dataloader = DataLoader(training_data, batch_size=BATCHSIZE, shuffle=True, drop_last=False, num_workers=4)
    valid_dataloader = DataLoader(validing_data, batch_size=BATCHSIZE, shuffle=False, drop_last=False, num_workers=4)
    
    NET = Network(len(inputszlist), inputszlist, hiddensz=1536, outsz=OUTSZ).to(DEVICE)
    # print(NET)
    optimizer = torch.optim.Adam(NET.parameters(), lr=1e-4, weight_decay=0)


    for epoch in range(EPOCH):
        data_enumerator = enumerate(train_dataloader)
        valid_enumerator = enumerate(valid_dataloader)
        loss_epoch = 0
        train_ans = []
        y_list = []
        NET.train()
        for step, (data1, data2, y) in data_enumerator:
            optimizer.zero_grad()
            data1 = tuple([data.to(DEVICE) for data in data1])
            data2 = tuple([data.to(DEVICE) for data in data2])

            y = y.float().to(DEVICE)
            out1 = NET(data1)
            out2 = NET(data2)
            loss, preds = LOSS(out1, out2, y, return_sim=True)
            # print(preds.shape)
            # print(y.shape)
            # preds = cosine(out1, out2)

            train_ans.extend(list(preds.detach().cpu().numpy()))
            y_list.extend(list(y.detach().cpu().numpy()))
            loss_epoch += loss.item()
            if step % 50 == 0:
                print(y)
                print(f"epoch:{epoch} \t , step:{step}\t loss:", loss.item())
            loss.backward()
            optimizer.step()

        if epoch >= START_INFERTEN_EPOCH:
            vid_emb = inference_one_epoch(test_dataloader, NET)
            vid_embedding_list.append(vid_emb)

        if not ALLIN:
            spearmanr = scipy.stats.spearmanr(train_ans, y_list).correlation
            info = f"epoch:{epoch}\t spearmanr:{spearmanr} \tloss_epoch:{loss_epoch / len(train_dataloader)}"
            print(colored(info, 'blue'))

        loss_epoch = 0
        val_ans = []
        y_list = []
        if not ALLIN:
            NET.eval()
            for step, (data1, data2, y) in valid_enumerator:
                optimizer.zero_grad()
                data1 = tuple([data.to(DEVICE) for data in data1])
                data2 = tuple([data.to(DEVICE) for data in data2])

                y = y.float().to(DEVICE)
                with torch.no_grad():
                    out1 = NET(data1)
                    out2 = NET(data2)
                    # print(final_emb1)
                    loss, preds = LOSS(out1, out2, y, return_sim=True)
                    val_ans.extend(list(preds.detach().cpu().numpy()))
                    y_list.extend(list(y.detach().cpu().numpy()))
                    loss_epoch += loss.item()
                if step % 10 == 0:
                    print(y)
                    print(f"valid epoch:{epoch} \t , step:{step}\t loss:", loss.item())

            spearmanr = scipy.stats.spearmanr(val_ans, y_list).correlation
            info = f"valid ::: epoch:{epoch}\t spearmanr:{spearmanr} \tloss_epoch:{loss_epoch / len(valid_dataloader)}"
            print(colored(info, 'green'))



(54319, 3)
(13580, 3)
tensor([0.0000, 0.9000, 0.0000,  ..., 0.8000, 0.0500, 1.0000], device='cuda:0')
epoch:0 	 , step:0	 loss: -0.5743523240089417
epoch:0	 spearmanr:0.7929489776222366 	loss_epoch:-0.8192177807029924
tensor([0.2500, 0.0500, 0.0000,  ..., 0.0000, 0.6500, 0.7000], device='cuda:0')
valid epoch:0 	 , step:0	 loss: -0.8446042537689209
valid ::: epoch:0	 spearmanr:0.8317599724967318 	loss_epoch:-0.8414620935916901
tensor([1.0000, 0.0000, 0.1000,  ..., 0.1500, 0.0500, 0.3500], device='cuda:0')
epoch:1 	 , step:0	 loss: -0.945759117603302
epoch:1	 spearmanr:0.909053062170239 	loss_epoch:-0.9475517021982294
tensor([0.2500, 0.0500, 0.0000,  ..., 0.0000, 0.6500, 0.7000], device='cuda:0')
valid epoch:1 	 , step:0	 loss: -0.8622879981994629
valid ::: epoch:1	 spearmanr:0.8466848866267687 	loss_epoch:-0.8572422206401825
tensor([0.0500, 0.2500, 0.8500,  ..., 0.0000, 1.0000, 0.0000], device='cuda:0')
epoch:2 	 , step:0	 loss: -0.9766716957092285
epoch:2	 spearmanr:0.9373619468164834 

KeyboardInterrupt: 

In [ ]:
# add CNN epoch:10	 spearmanr:0.8386718699764899 	loss_epoch:-0.8510410019329616
# add trans epoch:4	 spearmanr:0.8474135897243554 	loss_epoch:-0.8593714322362628
# add roberta ich
# SENet BN epoch:5	 spearmanr:0.8484653604794136 	loss_epoch:-0.8606224996703011
# BN SENet BN epoch:5	 spearmanr:0.8519170972032961 	loss_epoch:-0.8641518013817924
# BN SENet epoch:5	 spearmanr:0.8504814605848442 	loss_epoch:-0.86269554070064
# w/o SENet  epoch:5	 spearmanr:0.8484274529439838 	loss_epoch:-0.8610541905675616
# +++++ remove all dropout ｜ add BN  epoch:5	 spearmanr:0.8513013300677326 	loss_epoch:-0.8634524089949471
# BN SENet BN epoch:5	 spearmanr:0.8518227148627213 	loss_epoch:-0.8639950922557286
# MSE loss epoch:5	 spearmanr:0.8301060927771783 	loss_epoch:0.04277116911751883

# 2000 samples
# epoch:5	 spearmanr:0.8314429985271974 	loss_epoch:-0.8408079147338867


In [21]:
vid_embedding = merge_embedding(vid_embedding_list)

total 9 embeddings


100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 43027/43027 [00:11<00:00, 3744.34it/s]


In [22]:
path = 'two_stage/5foldSP_addyrq_mean'
import os

if not os.path.exists(path):
    os.makedirs(path)
output_json = path + '/result.json'
output_zip = path + '/result.zip'

import json
import joblib
from zipfile import ZIP_DEFLATED, ZipFile

with open(output_json, 'w') as f:
    json.dump(vid_embedding, f)
with ZipFile(output_zip, 'w', compression=ZIP_DEFLATED) as zip_file:
    zip_file.write(output_json)




##### 